In [1]:
%load_ext autoreload
%autoreload 2

In [92]:
import sys_pdf
import pdf_functions as pdf
import os

'''
Process a PDF using search dictionary
'''

def open_and_read(working_dir, file_name):# open a file and read
    try: 
        path_name = working_dir + "/" + file_name
        scanned_text = pdf.read_pdf(path_name)
    except:
        sys_pdf.sys_log_entry('read error', "Could not open file: " + path_name)
        sys_pdf.move_file(working_dir, "not_processed", file_name)
    return scanned_text, path_name


def format_text(scanned_text):
    # add the contents for each page into a list
    contents = pdf.text_to_list(scanned_text)

    # iterate over each page
    by_line = []
    for page in range(0, len(scanned_text.pages)):  # create a flat list of strings
        by_line += contents[page].split('\n')       # remove the line breaks
    
    return by_line


def process_pdfs(working_dir):
    file_counter = 0
     
    # for filename in working_dir
    for file_name in os.listdir(working_dir):
    
        # pull text from pdf file
        scanned_text, path_name = open_and_read(working_dir, file_name)
        
        # process the file
        try: 
            # make a list of text
            by_line = format_text(scanned_text)
            
            # open dictionary of search terms
            table_1_search, table_1_type = pdf.table_1_search_dict()
            table_2_search, table_2_type = pdf.table_2_search_dict()
        
            # create tables with pulled data
            table_1 = pdf.one_table(table_1_search, by_line)
            table_2 = pdf.many_table(table_2_search, by_line)  
        
            # 
            table_1_df = pdf.one_table_df(table_1)
            table_2_df = pdf.many_table_df(table_2)  # verifies that the length of each value list is the same in a many table
        
            # add a column of invoice numbers for each table_2 entry
            #table_2_df['invoice_number'] = table_1['invoice_number'][0]
            table_2_df.insert(0, 'invoice_number', table_1['invoice_number'][0])
        
            # format dates to standard YYYY-MM-DD
            formatted_date_str = pdf.format_date(table_1['invoice_date'][0], '%B %d, %Y')  # November 11, 2015
            name_base = table_1['company_name'][0].replace('.', '_') + '_' + formatted_date_str
        
            pdf.save_as_csv(table_1, table_1_df, name_base, 'overview')
            pdf.save_as_csv(table_1, table_2_df, name_base, 'products')
        
            # save the pdf in the 'processed' directory
            sys_pdf.move_file(working_dir, "processed", file_name)
            
            file_counter += 1
            
        except:
            sys_pdf.sys_log_entry('production error', "Could not properly analyze file: " + path_name)
            #  move pdf to error directory
            sys_pdf.move_file(working_dir, "not_processed", file_name)
    
    print(f"A total of {file_counter} files were processed.")

In [70]:
# Move PDF files to working directory
source_dir = "pdf_inbox"
working_dir = "working_dir"
files_present = sys_pdf.to_working_dir(source_dir, working_dir)

if files_present:   # only move forward with formatting if there are files to process
    process_pdfs(working_dir)

no files There are no files in the pdf_inbox directory.


In [94]:
'''
Build a search dictionary.
'''
import pandas as pd

# Move PDF files to working directory
source_dir = "NEW_PDF_style"
working_dir = "working_dir"
search_file_name = "amazon111924.xlsx"

# checks directory if is a new PDF file to be formatted, if s move to working dir
files_present = sys_pdf.to_working_dir(source_dir, working_dir)

if files_present:
    # open excel file as a pandas dataframe
    guide = pd.read_excel("search_guides/"+search_file_name)
    
    # table headers from the guide
    overview_table_headers = [ 'Invoice date', 'Invoice number', 'Company name', 'Invoice total' ]
    products_table_headers = [ 'Product name', 'Product company', 'Product amount' ]

    # the name of the pdf file from search guide
    pdf_file = guide['File name of PDF'][0]
    
    # open and pull text from pdf file
    scanned_text, path_name = open_and_read(working_dir, pdf_file)

    # prep the text strings for scanning
    by_line = format_text(scanned_text)

    # iterate over each table header
    #for header in overview_table_headers:
    header = guide['Invoice number'][0]
    
    # iterate over each line in the list of text strings
    for line in by_line:
        # is there a match between the search string and the line?
        # if so return the match
        #matching_string = pdf.find_string(line, header, position, del_chars)

    if matching_string != None:
        # clean up the string
        clean_string = clean_text(matching_string)

move Files in inbox: 1. Moved to working dir: 1. 
The file has 2 pages.


In [100]:
guide['Invoice number'][0]

'104-1523806-7547463'

In [136]:
pos=0
header = guide['Invoice number'][0]
len_target_text = len(header)
for line in by_line:
    char_location = line.find(header)
    if char_location >= 0: 
        the_line = line
        pos=char_location
        break


In [137]:
the_line

'Details for Orde r #104-1523806-7547463 '

In [138]:
pos

20

In [132]:
# The search term 
front = the_line[0:pos]

back = the_line[pos+len_search_str:]

In [139]:
the_line[0:pos]

'Details for Orde r #'

In [3]:
import os

In [4]:
os.environ

environ{'TERM_PROGRAM': 'Apple_Terminal',
        'SHELL': '/bin/zsh',
        'TERM': 'xterm-color',
        'TMPDIR': '/var/folders/mx/9z5jxmj95z98yvdqsby2m0ph0000gp/T/',
        'TERM_PROGRAM_VERSION': '452',
        'TERM_SESSION_ID': '5F2CEE61-B53E-4380-8E80-F000D04C16F6',
        'USER': 'troy',
        'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.B4ZOau1HpN/Listeners',
        'PATH': '/Users/troy/google-cloud-sdk/bin:/usr/local/Caskroom/miniforge/base/envs/pdf/bin:/usr/local/Caskroom/miniforge/base/condabin:/Applications/Postgres.app/Contents/Versions/15/bin:/opt/homebrew/bin:/Users/troy/bin:/usr/local/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/Library/Apple/usr/bin',
        'LaunchInstanceID': '13DFBBB6-E81F-4770-AA

In [11]:


# Access environment variables
open_ai_key = os.getenv("OPENAI_API_KEY")

In [ ]:
import dotenv
import os
import openai

# Load environment variables from the .env file
dotenv.load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")
messages = [ {"role": "system", 
              "content": "You are a intelligent assistant."} ]
while True:
    message = input("User : ")
    if message:
        messages.append(
            {"role": "user", "content": message},
        )
        chat = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=messages
        )
    reply = chat.choices[0].message.content
    print(f"ChatGPT: {reply}")
    messages.append({"role": "assistant", "content": reply})